# 7. Tracking Costs in Databricks

Cost management is crucial for maximizing the value of your Databricks investment. This notebook demonstrates how to monitor usage, analyze spending patterns, and optimize costs across your workspace.

## Learning Objectives
- Access and navigate the Usage Dashboard
- Query system tables for detailed cost analysis
- Set up cost monitoring and alerts
- Implement cost optimization strategies

## Key Features Covered
- **Usage Dashboard**: Real-time cost monitoring and trends
- **System Tables**: Granular billing data for analysis
- **Resource Optimization**: Rightsizing clusters and warehouses
- **Cost Allocation**: Tracking costs by team, project, or use case


## Cost Monitoring in Databricks SQL (Video)
[![Video Thumbnail](https://img.youtube.com/vi/v0CsjJ-F-zs/0.jpg)](https://www.youtube.com/watch?v=v0CsjJ-F-zs)

## 1. Usage Dashboard Overview

The Usage Dashboard provides a comprehensive view of your Databricks consumption and costs:

### Key Metrics
- **DBU Consumption**: Total compute units consumed
- **Cost Trends**: Daily, weekly, and monthly spending patterns
- **Resource Breakdown**: Costs by workspace, cluster, and job
- **Usage Patterns**: Peak usage times and resource utilization

### Accessing the Dashboard
1. Navigate to your Databricks account console
2. Select "Usage" from the left sidebar
3. Choose your desired time range and filters
4. Explore different views: Overview, Detailed Usage, and Billing

**Note**: Account admin privileges are required to access usage data.

In [0]:
SHOW TABLES IN system.billing;

In [0]:
%sql
-- 2. System Tables for Cost Analysis
-- Query actual billing data using Databricks system tables

-- View recent usage by workspace with joined list_prices for accurate cost calculation
SELECT
    u.billing_origin_product,
    u.usage_date,
    SUM(u.usage_quantity) as usage_quantity,
    SUM(u.usage_quantity * lp.pricing.effective_list.default) as total_dollar_cost
FROM system.billing.usage u
JOIN system.billing.list_prices lp
    ON lp.sku_name = u.sku_name
    AND u.usage_end_time >= lp.price_start_time
    AND (lp.price_end_time IS NULL OR u.usage_end_time < lp.price_end_time)
WHERE
    month(u.usage_date) = month(NOW())
    AND year(u.usage_date) = year(NOW())
GROUP BY u.billing_origin_product, u.usage_date

In [0]:
%sql
-- Daily cost trends over the last 30 days using effective list prices
SELECT 
    u.usage_date,
    SUM(u.usage_quantity * lp.pricing.effective_list.default) as daily_cost,
    COUNT(DISTINCT u.workspace_id) as active_workspaces
FROM system.billing.usage u
JOIN system.billing.list_prices lp
    ON lp.sku_name = u.sku_name
    AND u.usage_end_time >= lp.price_start_time
    AND (lp.price_end_time IS NULL OR u.usage_end_time < lp.price_end_time)
WHERE u.usage_date >= current_date() - INTERVAL 30 days
GROUP BY u.usage_date
ORDER BY u.usage_date DESC;

In [0]:
%sql
-- Top consuming resources by SKU type
SELECT 
    u.sku_name,
    SUM(u.usage_quantity) as total_dbus,
    SUM(u.usage_quantity * lp.pricing.effective_list.default) as total_cost,
    AVG(u.usage_quantity * lp.pricing.effective_list.default) as avg_daily_cost
FROM system.billing.usage u
JOIN system.billing.list_prices lp
    ON lp.sku_name = u.sku_name
    AND u.usage_end_time >= lp.price_start_time
    AND (lp.price_end_time IS NULL OR u.usage_end_time < lp.price_end_time)
WHERE u.usage_date >= current_date() - INTERVAL 7 days
GROUP BY u.sku_name
ORDER BY total_cost DESC;

## 3. Cost Optimization Strategies

### Cluster Management
- **Auto Termination**: Set appropriate idle timeouts
- **Rightsizing**: Match cluster size to workload requirements
- **Spot Instances**: Use for fault-tolerant workloads
- **Pool Management**: Reuse instances across jobs

### SQL Warehouse Optimization
- **Auto Scaling**: Enable based on query load
- **Serverless SQL**: Consider for variable workloads
- **Query Optimization**: Improve query performance to reduce runtime

### Monitoring and Alerts
- Set up budget alerts in the Usage Dashboard
- Create custom dashboards for team-specific cost tracking
- Implement automated reporting for stakeholders

In [0]:
%sql
-- 4. Cost Analysis by Tag (if tagging is implemented)
-- This helps with cost allocation and chargeback

SELECT 
    u.custom_tags['department'] as department,
    u.custom_tags['project'] as project,
    SUM(u.usage_quantity * lp.pricing.effective_list.default) as total_cost,
    COUNT(*) as usage_records
FROM system.billing.usage u
JOIN system.billing.list_prices lp
    ON lp.sku_name = u.sku_name
    AND u.usage_end_time >= lp.price_start_time
    AND (lp.price_end_time IS NULL OR u.usage_end_time < lp.price_end_time)
WHERE u.usage_date >= current_date() - INTERVAL 30 days
    AND u.custom_tags IS NOT NULL
GROUP BY u.custom_tags['department'], u.custom_tags['project']
ORDER BY total_cost DESC;

## 5. Setting Up Cost Monitoring

### Dashboard Creation
1. Create a new dashboard in Databricks SQL
2. Add visualizations for:
   - Daily cost trends
   - Top spending workspaces
   - Resource utilization patterns
   - Budget vs. actual spending

### Automated Alerts
```python
# Example: Cost threshold alert using Databricks Jobs
def check_daily_costs():
    daily_cost = spark.sql("""
        SELECT SUM(usage_quantity * list_price) as cost
        FROM system.billing.usage
        WHERE usage_date = current_date()
    """).collect()[0]['cost']
    
    if daily_cost > 1000:  # $1000 threshold
        send_alert(f"Daily costs exceeded threshold: ${daily_cost:.2f}")
```

### Best Practices
- Review costs weekly with stakeholders
- Implement cost allocation tags consistently
- Set up automated cost reports
- Regular rightsizing reviews
- Monitor unused resources

## Key Takeaways

✅ **Usage Dashboard**: Your central hub for cost monitoring and analysis  
✅ **System Tables**: Provide granular data for custom cost analytics  
✅ **Optimization**: Regular review and rightsizing saves significant costs  
✅ **Monitoring**: Proactive alerts prevent budget overruns  
✅ **Allocation**: Tags enable accurate cost attribution and chargeback  

## Next Steps
- Set up regular cost review meetings
- Implement tagging strategy for cost allocation
- Create automated cost monitoring dashboards
- Establish cost optimization workflows

## Resources
- [Databricks Cost Management Guide](https://docs.databricks.com/administration-guide/account-settings/usage.html)
- [System Tables Reference](https://docs.databricks.com/administration-guide/system-tables/index.html)
- [Billing APIs Documentation](https://docs.databricks.com/dev-tools/api/latest/account.html#tag/Billing)